<a href="https://colab.research.google.com/github/C4viar/PPD_MSLD_2019_2020_LL-AL-MT/blob/master/Draft_code/projet_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Montage du drive

In [1]:
# Montage du drive

from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


# Contexte


Des modèles de langage du français ont tout récemment été publiés (CamemBERT, FlauBERT C’est positif car même des outils avancés comme Spacy sont notoirement encore peu adaptés à traiter certaines langues dont le français. Il reste à évaluer ce que ces modèles peuvent vraiment apporter notamment ne ce qui concerne la reconnaissances d’entités nommées.


# Travail a réaliser

Réaliser des expériences de reconnaissance d’entités nommées sur le jeu de donneés Europeana Newspapers French en utilisant tous les modèles disponibles pour cela dans la bibliothèque Simple Transformers. Les adresses à consulter sont :
* https://github.com/ThilinaRajapakse/simpletransformers
* http://api.bnf.fr/europeana-newspapers-french-ner-ground-truth

# Rendu

Un code clair et commenté dans un notebook.
Un rapport décrivant les méthodes/métriques d’évaluation (basées sur seqeval dont une description détaillée sera donnée) et présentant/comparant/discutant les résultats obtenus.


# Seqeval 

https://pypi.org/project/seqeval/

"seqeval is a Python framework for sequence labeling evaluation. seqeval can evaluate the performance of chunking tasks such as named-entity recognition, part-of-speech tagging, semantic role labeling and so on."

# Installation simple transformers

In [2]:
pip install simpletransformers

     |████████████████████████████████| 174kB 8.7MB/s 
     |████████████████████████████████| 3.8MB 9.6MB/s 
     |████████████████████████████████| 645kB 53.7MB/s 
     |████████████████████████████████| 204kB 59.2MB/s 
     |████████████████████████████████| 1.0MB 45.8MB/s 
     |████████████████████████████████| 890kB 56.7MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=e4d1fa3337d2f574a04a130462b095586b5352bd0e905f5260f30942d5ad6f71
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=35df6c15aea78bb997794ced494f99bfe0f3c9749f543ed2c91b11b937d4f5c0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built seqeval sacremoses


In [3]:
pip install wandb

     |████████████████████████████████| 1.4MB 8.5MB/s 
     |████████████████████████████████| 460kB 58.0MB/s 
     |████████████████████████████████| 112kB 57.2MB/s 
     |████████████████████████████████| 102kB 14.6MB/s 
     |████████████████████████████████| 102kB 13.7MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 71kB 10.7MB/s 
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=26bccd6c2684fb2fdf4372c5ab96de729dd2e261b9f6a49a2fbfa7bb543e4656
  Stored in directory: /root/.cache/pip/wheels/ce/0e/7b/58a8a5268655b3ad74feef5aa97946f0addafb3cbb6bd2da23
  Created wheel for watchdog: filename=watchdog-0.10.2-cp36-none-any.whl size=73605 sha256=9963dd397ad329caf8883659c4c437c5acf9733cdec9e828177fc0606d112c4c
  Stored in directory: /root/.cache/pip/wheels/bc/ed/6c/028dea90d31b359cd2a7c8b0da4db80e41d24a59614154072e
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=

# Installation APEX

In [4]:
! git clone https://github.com/NVIDIA/apex

Cloning into 'apex'...
remote: Enumerating objects: 6742, done.
remote: Total 6742 (delta 0), reused 0 (delta 0), pack-reused 6742
Receiving objects: 100% (6742/6742), 13.73 MiB | 14.62 MiB/s, done.
Resolving deltas: 100% (4505/4505), done.


In [5]:
cd apex

/content/apex


In [6]:
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-qe4ys46p
Created temporary directory: /tmp/pip-req-tracker-vplig9gi
Created requirements tracker '/tmp/pip-req-tracker-vplig9gi'
Created temporary directory: /tmp/pip-install-tb7jwmqy
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-6w9ehwhz
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-vplig9gi'
    Running setup.py (path:/tmp/pip-req-build-6w9ehwhz/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info
    torch.__version__  =  1.5.0+cu101
    running egg_info
    creating /tmp/pip-req-build-6w9ehwhz/pip-egg-info/apex.egg-info
    writing /tmp/pip-req-build-6w9ehwhz/pip-egg-info/apex.egg-info/PKG-I

# NER 
https://simpletransformers.ai/docs/ner-specifics/

"The goal of Named Entity Recognition is to locate and classify named entities in a sequence. The named entities are pre-defined categories chosen according to the use case such as names of people, organizations, places, codes, time notations, monetary values, etc. Essentially, NER aims to assign a class to each token (usually a single word) in a sequence. Because of this, NER is also referred to as token classification."

**Fonctionnement avec Simple Transformer**

* Initialize a NERModel
* Train the model with train_model()
* Evaluate the model with eval_model()
* Make predictions on (unlabelled) data with predict()

**Modèles disponibles à fournir à model_type** : https://huggingface.co/models

* BERT :	*bert*
* CamemBERT :	*camembert*
* RoBERTa :	*roberta*
* DistilBERT :	*distilbert*
* ELECTRA	: *electra*
* XLM-RoBERTa	: *xlmroberta*

In [0]:
from simpletransformers.ner import NERModel

# to custom labels
custom_labels = ["O", "B-SPELL", "I-SPELL", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-PLACE", "I-PLACE"]


model = NERModel('bert', 'bert-base-cased', args={'overwrite_output_dir': True, 'reprocess_input_data': True}, labels=custom_labels)

In [0]:
from simpletransformers.ner import NERModel
import pandas as pd
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Creating train_df  and eval_df for demonstration
train_data = [
    [0, 'Simple', 'B-MISC'], [0, 'Transformers', 'I-MISC'], [0, 'started', 'O'], [1, 'with', 'O'], [0, 'text', 'O'], [0, 'classification', 'B-MISC'],
    [1, 'Simple', 'B-MISC'], [1, 'Transformers', 'I-MISC'], [1, 'can', 'O'], [1, 'now', 'O'], [1, 'perform', 'O'], [1, 'NER', 'B-MISC']
]
train_df = pd.DataFrame(train_data, columns=['sentence_id', 'words', 'labels'])

eval_data = [
    [0, 'Simple', 'B-MISC'], [0, 'Transformers', 'I-MISC'], [0, 'was', 'O'], [1, 'built', 'O'], [1, 'for', 'O'], [0, 'text', 'O'], [0, 'classification', 'B-MISC'],
    [1, 'Simple', 'B-MISC'], [1, 'Transformers', 'I-MISC'], [1, 'then', 'O'], [1, 'expanded', 'O'], [1, 'to', 'O'], [1, 'perform', 'O'], [1, 'NER', 'B-MISC']
]
eval_df = pd.DataFrame(eval_data, columns=['sentence_id', 'words', 'labels'])

# Create a NERModel
model = NERModel('bert', 'bert-base-cased', args={'overwrite_output_dir': True, 'reprocess_input_data': True})

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, predictions = model.eval_model(eval_df)

# Predictions on arbitary text strings
predictions, raw_outputs = model.predict(["Some arbitary sentence"])

print(predictions)

INFO:simpletransformers.ner.ner_model: Converting to features started.



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 2.324800Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0



/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:114: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.ner.ner_model: Training of bert model complete. Saved to outputs/.
INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 2.2720589637756348, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0}


INFO:simpletransformers.ner.ner_model: Converting to features started.



[[{'Some': 'I-MISC'}, {'arbitary': 'I-MISC'}, {'sentence': 'I-MISC'}]]


In [0]:
import pandas as pd 

In [14]:
cd drive/My Drive/PPD-NER

/content/drive/My Drive/PPD-NER


In [0]:
import os
files = os.listdir("./data")

In [0]:
l_bug = []
l_ok = []
df = pd.DataFrame()
from pandas.io.parsers import ParserError
for i in files :
  try:
    df = df.append(pd.read_csv("./data/"+i, sep = "\t", quoting = 3, header = None))
    l_ok.append(i)
  except ParserError as err :
    l_bug.append(i)

In [0]:
shape_df = pd.DataFrame(columns = ["file", "rows", "cols"])
for i in files : 
  tmp = pd.read_csv("./data/"+i, sep = "\t", quoting = 3, header = None)
  rows = tmp.shape[0]
  cols = tmp.shape[1]
  dict_tmp = {
      "file" : i ,
      "rows" : rows,
      "cols" : cols
  }
  shape_df = shape_df.append(dict_tmp, ignore_index=True)

In [52]:
shape_df

,file,rows,cols
0,xaa.L.jp.tag,1000,2
1,xba.L.tag,1000,2
2,xbb.L.tag,1000,2
3,xbc.L.tag,1000,2
4,xbd.H.tag,1000,2
...,...,...,...
207,axdj.M.a.tag,1000,2
208,axdk.H.a.tag,1000,2
209,axdl.H.a.tag,1000,2
210,axdm.H.a.tag,1000,2


In [0]:
df.columns = ["words", "labels"]

In [54]:
df

,words,labels
0,:,O
1,:,O
2,:,O
3,:,O
4,:,O
...,...,...
995,l',O
996,orateur,O
997,dont,O
998,le,O


In [0]:
df = df.reset_index(drop = True )

In [0]:
j = 0
df["sentence_id"] = ""
for i in range(df.shape[0]):
  if (df.loc[i, "words"] in ([".", "!", "?"])):
    df.loc[i, "sentence_id"] = j
    j = j+1
  else :
    df.loc[i, "sentence_id"] = j

In [57]:
df

,words,labels,sentence_id
0,:,O,0
1,:,O,0
2,:,O,0
3,:,O,0
4,:,O,0
...,...,...,...
211995,l',O,12867
211996,orateur,O,12867
211997,dont,O,12867
211998,le,O,12867


In [0]:
df = df[["sentence_id", "words", "labels"]]

In [68]:
import random
randomlist = []
for i in range(0,10293):
  n = random.randint(0, df["sentence_id"].nunique())
  randomlist.append(n)
print(randomlist)

[4260, 8030, 9277, 39, 2018, 6803, 3462, 767, 1148, 11291, 4891, 8727, 3637, 8228, 4093, 6173, 8329, 7527, 3681, 10104, 11871, 3341, 7543, 1454, 12332, 9875, 8620, 7072, 2269, 8028, 6397, 5265, 1873, 6075, 11857, 636, 2111, 11489, 8150, 2994, 11293, 9662, 6027, 12398, 7446, 7175, 7770, 3630, 5856, 3185, 3856, 12820, 10480, 9412, 3663, 9871, 2275, 8378, 5645, 193, 4786, 11996, 5959, 3968, 8342, 6819, 3489, 5555, 12759, 7396, 6435, 8026, 11896, 4778, 5234, 887, 12227, 10774, 6465, 6001, 9173, 10221, 9310, 1737, 3657, 10695, 7116, 8208, 8522, 2584, 51, 7290, 4585, 2264, 11830, 11851, 10582, 5449, 12369, 6964, 1641, 12219, 11946, 3323, 12778, 10257, 12264, 7563, 4133, 7245, 1953, 5692, 2353, 2389, 1695, 12738, 1722, 7110, 10711, 917, 8135, 9638, 8601, 993, 4744, 7935, 2675, 105, 1765, 6255, 7488, 10699, 1367, 3367, 7152, 5341, 6432, 8008, 8836, 2425, 7403, 4010, 7452, 8384, 5499, 7883, 9415, 8596, 6432, 9410, 4635, 8198, 6516, 5146, 5815, 9565, 7134, 467, 3140, 7256, 3217, 7508, 2, 7951, 5

In [0]:
tmp = df[df["sentence_id"].isin(randomlist)]

In [0]:
tags = df["labels"].unique().tolist()

In [0]:
df.to_csv("./out/df.csv", index = False, sep = "\t" )

In [0]:
from simpletransformers.ner import NERModel
import pandas as pd
import logging
import numpy as np
from sklearn.model_selection import train_test_split

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Creating train_df  and eval_df for demonstration

train_df = df[df["sentence_id"].isin(randomlist)]

eval_df = df[~df["sentence_id"].isin(randomlist)]

# Create a NERModel
model = NERModel('bert', 'bert-base-cased', args={'overwrite_output_dir': True, 'reprocess_input_data': True}, labels=tags)

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, predictions = model.eval_model(eval_df)

# Predictions on arbitary text strings
predictions, raw_outputs = model.predict(["Some arbitary sentence"])

print(predictions)

INFO:filelock:Lock 139930469081608 acquired on /root/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391.lock


INFO:filelock:Lock 139930469081608 released on /root/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391.lock
INFO:filelock:Lock 139927962170424 acquired on /root/.cache/torch/transformers/d8f11f061e407be64c4d5d7867ee61d1465263e24085cfa26abf183fdc830569.3fadbea36527ae472139fe84cddaa65454d7429f12d543d80bfc3ad70de55ac2.lock


INFO:filelock:Lock 139927962170424 released on /root/.cache/torch/transformers/d8f11f061e407be64c4d5d7867ee61d1465263e24085cfa26abf183fdc830569.3fadbea36527ae472139fe84cddaa65454d7429f12d543d80bfc3ad70de55ac2.lock


INFO:filelock:Lock 139927948532088 acquired on /root/.cache/torch/transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1.lock


INFO:filelock:Lock 139927948532088 released on /root/.cache/torch/transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1.lock


...

Running loss: 0.068535

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 2.084645


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.ner.ner_model: Training of bert model complete. Saved to outputs/.
INFO:simpletransformers.ner.ner_model: Converting to features started.


In [0]:
predictions

[[{'Some': 'O'}, {'arbitary': 'O'}, {'sentence': 'O'}]]